In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper
os.environ["SERPAPI_API_KEY"] = ""

GoogleFinanceTool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())

In [8]:
tool.run("Google")

'\nQuery: Google\nstock: GOOG:NASDAQ\nprice: $165.57\npercentage: 0.56\nmovement: Up\nus: price = 37903.29, movement = Up\neurope: price = 17932.17, movement = Down\nasia: price = 38236.07, movement = Down\n'

In [11]:
tools=[GoogleFinanceTool]

In [12]:
tools

[GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper(serp_search_engine=<class 'serpapi.serp_api_client.SerpApiClient'>, serp_api_key=SecretStr('**********')))]

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [14]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [16]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [17]:
agent_executor.invoke({"input":"Tell me stock prce history of Meta"})




> Entering new AgentExecutor chain...

Invoking: `google_finance` with `Meta stock price history`



Query: Meta stock price history
No summary information
us: price = 37903.29, movement = Up
europe: price = 17932.17, movement = Down
asia: price = 38236.07, movement = Down
The stock price history of Meta is as follows:
- In the US, the price is $37,903.29 with an upward movement.
- In Europe, the price is $17,932.17 with a downward movement.
- In Asia, the price is $38,236.07 with a downward movement.

> Finished chain.


{'input': 'Tell me stock prce history of Meta',
 'output': 'The stock price history of Meta is as follows:\n- In the US, the price is $37,903.29 with an upward movement.\n- In Europe, the price is $17,932.17 with a downward movement.\n- In Asia, the price is $38,236.07 with a downward movement.'}